In [48]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [49]:
import pandas as pd

In [50]:
df1 = pd.read_excel("/content/drive/MyDrive/Tacos_data/MIZO--- ENGLISH---- MIZO.xlsx")

In [51]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Text       800 non-null    object
 1   Sentiment  800 non-null    object
dtypes: object(2)
memory usage: 12.6+ KB


In [52]:
df1.head()

,Text,Sentiment
0,"He video hi en suh, hun khawhralna a ni.",NEGATIVE
1,I video te hi ka duh lo khawp mai.,NEGATIVE
2,i video ah hian information mak tak tak i nei a.,NEGATIVE
3,Engvangin nge i video hi a chhiat em em?,NEGATIVE
4,I video te hi a tha tawk ka ti lo,NEGATIVE


In [53]:
df2 = pd.read_excel("/content/drive/MyDrive/Tacos_data/English_Tacos_Train.xlsx")

In [54]:
df2.head()

,Text,Sentiment
0,Basically there's a family where a little boy ...,NEGATIVE
1,"This show was an amazing, fresh & innovative i...",NEGATIVE
2,Encouraged by the positive comments about this...,NEGATIVE
3,Phil the Alien is one of those quirky films wh...,NEGATIVE
4,I saw this movie when I was about 12 when it c...,NEGATIVE


In [55]:
#df = df1

In [56]:
df =  pd.concat([df1, df2])

In [57]:
df.head()

,Text,Sentiment
0,"He video hi en suh, hun khawhralna a ni.",NEGATIVE
1,I video te hi ka duh lo khawp mai.,NEGATIVE
2,i video ah hian information mak tak tak i nei a.,NEGATIVE
3,Engvangin nge i video hi a chhiat em em?,NEGATIVE
4,I video te hi a tha tawk ka ti lo,NEGATIVE


In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1600 entries, 0 to 799
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Text       1600 non-null   object
 1   Sentiment  1600 non-null   object
dtypes: object(2)
memory usage: 37.5+ KB


In [59]:
#converting the labels to numbers
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['labels'] = le.fit_transform(df.Sentiment.values)

In [60]:
df.head()

,Text,Sentiment,labels
0,"He video hi en suh, hun khawhralna a ni.",NEGATIVE,0
1,I video te hi ka duh lo khawp mai.,NEGATIVE,0
2,i video ah hian information mak tak tak i nei a.,NEGATIVE,0
3,Engvangin nge i video hi a chhiat em em?,NEGATIVE,0
4,I video te hi a tha tawk ka ti lo,NEGATIVE,0


In [61]:
from sklearn.utils import shuffle
df = shuffle(df)

In [62]:
positive_labels = (df['labels'] == 1).sum()
negative_labels = (df['labels'] == 0).sum()

In [63]:
print(f"Number of positive labels: {positive_labels}")
print(f"Number of negative labels: {negative_labels}")

Number of positive labels: 800
Number of negative labels: 800


In [64]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels_str, val_labels_str = train_test_split(list(df['Text']), list(df['Sentiment']), test_size=.2)

In [65]:
!pip install sentencepiece

In [66]:
!pip install transformers

In [67]:
!pip install transformers accelerate

In [68]:
!pip install torch==1.11.00.676617

ERROR: Could not find a version that satisfies the requirement torch==1.11.00.676617 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0)
ERROR: No matching distribution found for torch==1.11.00.676617


In [69]:
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
import torch

tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
model = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base',num_labels=2)
#print(model)

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [70]:
import torch

In [71]:
class spanish_Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_labels = le.fit_transform(train_labels_str)
val_labels = le.transform(val_labels_str)

train_dataset = spanish_Dataset(train_encodings, train_labels)
val_dataset = spanish_Dataset(val_encodings, val_labels)

In [72]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='micro')
    acc = accuracy_score(labels, preds)
    return {'Accuracy': acc,
            'F1': f1,
            'Precision': precision,
            'Recall': recall
           }

In [73]:
from transformers import EarlyStoppingCallback, IntervalStrategy,Trainer, TrainingArguments

In [74]:
!pip install --upgrade accelerate

In [75]:
!pip install accelerate -U

In [76]:
!pip install transformers[torch]

In [77]:
!pip install accelerate>=0.21.0

In [78]:
!pip install --upgrade accelerate


In [79]:
import torch
torch.cuda.empty_cache()

In [80]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [81]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=5,              # total number of training epochs 3,6,7,8
    per_device_train_batch_size=15,  # batch size per device during training
    per_device_eval_batch_size=15,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    do_eval=True,
    evaluation_strategy="steps",
    load_best_model_at_end=True,
    eval_steps=10
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset= val_dataset,
    compute_metrics=compute_metrics, # evaluation dataset
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
10,0.709400,0.690504,0.528125,0.528125,0.528125,0.528125
20,0.702000,0.690185,0.531250,0.531250,0.531250,0.531250
30,0.688300,0.690103,0.556250,0.556250,0.556250,0.556250
40,0.684100,0.689901,0.559375,0.559375,0.559375,0.559375
50,0.683400,0.685084,0.521875,0.521875,0.521875,0.521875
60,0.710500,0.685050,0.628125,0.628125,0.628125,0.628125
70,0.681600,0.684391,0.637500,0.637500,0.637500,0.637500
80,0.679000,0.668906,0.615625,0.615625,0.615625,0.615625
90,0.679300,0.653101,0.675000,0.675000,0.675000,0.675000
100,0.683000,0.662920,0.721875,0.721875,0.721875,0.721875


TrainOutput(global_step=430, training_loss=0.5033375396284946, metrics={'train_runtime': 1077.3739, 'train_samples_per_second': 5.94, 'train_steps_per_second': 0.399, 'total_flos': 1683910754304000.0, 'train_loss': 0.5033375396284946, 'epoch': 5.0})

In [82]:
metrics=trainer.evaluate()

In [83]:
metrics

{'eval_loss': 0.4628840982913971,
 'eval_Accuracy': 0.846875,
 'eval_F1': 0.846875,
 'eval_Precision': 0.846875,
 'eval_Recall': 0.846875,
 'eval_runtime': 10.5531,
 'eval_samples_per_second': 30.323,
 'eval_steps_per_second': 2.085,
 'epoch': 5.0}

In [84]:
test_df = pd.read_excel("/content/drive/MyDrive/Tacos_data/mizo_data_tacos_test_final(1).xlsx")

In [85]:
len(test_df)

200

In [86]:
test_df.head()

,Text,Sentiment
0,"He hla hi hmangaihna lamah te, min kawngah te ...",POSITIVE
1,He track-a guitar hnathawh inzawmkhawm leh sou...,POSITIVE
2,Genre hrang hrang inzawmkhawmna famkim ani . H...,POSITIVE
3,"A hla thu hi hla thu a ni a, a thluk pawh a ma...",POSITIVE
4,Positivity pholang thei hla tichaktu. Artist-i...,POSITIVE


In [87]:
test_texts = list(test_df['Text'])

In [88]:
test_labels = le.transform(test_df['Sentiment'])

In [89]:
test_encodings = tokenizer(test_texts, truncation=True, padding=True)
test_dataset = spanish_Dataset(test_encodings, test_labels)

In [90]:
# here we are evaluating a model using the Trainer object trainer on a test dataset.
test_metrics = trainer.evaluate(test_dataset)

In [91]:
test_metrics

{'eval_loss': 0.841168224811554,
 'eval_Accuracy': 0.7,
 'eval_F1': 0.7,
 'eval_Precision': 0.7,
 'eval_Recall': 0.7,
 'eval_runtime': 7.1326,
 'eval_samples_per_second': 28.04,
 'eval_steps_per_second': 1.963,
 'epoch': 5.0}